In [1]:
!pip install "protobuf==3.20.*" --force-reinstall
!pip install -q "monai[nibabel]" --no-deps


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 5.5 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
onnx 1.18.0 requires protobuf>=4.25.1, but you have protobuf 3.20.3 which is incompatible.
a2a-sdk 0.3.10 requires protobuf>=5.29.5, but you have protobuf 3.20.3 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
tensorflow-me

In [ ]:
import os, random, numpy as np, nibabel as nib, time
from tqdm.auto import tqdm
from glob import glob

import torch, torch.nn as nn
from torch.utils.data import DataLoader

import monai
from monai.transforms import (
    LoadImaged, EnsureChannelFirstd, Orientationd, Spacingd,
    CropForegroundd, NormalizeIntensityd, ResizeWithPadOrCropd,
    ConcatItemsd, EnsureTyped
)
from monai.data import Dataset, CacheDataset
from monai.networks.nets import SwinUNETR
from monai.losses import DiceCELoss, FocalLoss, TverskyLoss
from monai.inferers import sliding_window_inference

print("MONAI:", monai.__version__)
print("Torch:", torch.__version__)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


<frozen importlib._bootstrap_external>:1241: FutureWarning: The cuda.cudart module is deprecated and will be removed in a future release, please switch to use the cuda.bindings.runtime module instead.
2025-12-06 07:27:23.126093: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765006043.292541      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765006043.346300      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
BASE = "/kaggle/input/brats20-dataset-training-validation"

TRAIN_DIR = f"{BASE}/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData"
VAL_DIR   = f"{BASE}/BraTS2020_ValidationData/MICCAI_BraTS2020_ValidationData"
TEST_DIR  = VAL_DIR  # No labels provided, will use for inference only

IMG_SIZE = (96, 96, 96)
IN_CHANNELS = 4
OUT_CHANNELS = 4   # 0,1,2,3 (we remap 4->3)
BATCH_SIZE = 2
EPOCHS = 2
LR = 1e-4

OUTDIR = "/kaggle/working/swin_unetr_run"
CKPT_DIR = f"{OUTDIR}/checkpoints"; os.makedirs(CKPT_DIR, exist_ok=True)
PRED_DIR = f"{OUTDIR}/predictions"; os.makedirs(PRED_DIR, exist_ok=True)

SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)

print("OUTPUT DIR:", OUTDIR)


In [ ]:
def load_brats_train(root):
    items = []
    for case in sorted(os.listdir(root)):
        folder = f"{root}/{case}"

        # Skip junk files / CSV / non-directories
        if not os.path.isdir(folder): 
            continue
        if case.endswith(".csv") or case.startswith("."):
            continue
        
        cid = case
        
        f = f"{folder}/{cid}_flair.nii"
        t1 = f"{folder}/{cid}_t1.nii"
        t1ce = f"{folder}/{cid}_t1ce.nii"
        t2 = f"{folder}/{cid}_t2.nii"
        seg = f"{folder}/{cid}_seg.nii"

        # Make sure ALL 4 modalities + seg exist
        if os.path.exists(f) and os.path.exists(t1) and os.path.exists(t1ce) and os.path.exists(t2) and os.path.exists(seg):
            items.append({"id": cid, "flair": f, "t1": t1, "t1ce": t1ce, "t2": t2, "seg": seg})
    
    return items


def load_brats_test(root):
    items = []
    for case in sorted(os.listdir(root)):
        folder = f"{root}/{case}"
        if not os.path.isdir(folder): 
            continue
        if case.endswith(".csv") or case.startswith("."):
            continue

        cid = case
        f = f"{folder}/{cid}_flair.nii"
        t1 = f"{folder}/{cid}_t1.nii"
        t1ce = f"{folder}/{cid}_t1ce.nii"
        t2 = f"{folder}/{cid}_t2.nii"

        if os.path.exists(f) and os.path.exists(t1) and os.path.exists(t1ce) and os.path.exists(t2):
            items.append({"id": cid, "flair": f, "t1": t1, "t1ce": t1ce, "t2": t2})
    return items


train_files = load_brats_train(TRAIN_DIR)
test_files  = load_brats_test(TEST_DIR)

print("Train cases:", len(train_files))
print("Test cases:", len(test_files))


In [ ]:
train_transforms = monai.transforms.Compose([
    LoadImaged(keys=["flair","t1","t1ce","t2","seg"]),
    EnsureChannelFirstd(keys=["flair","t1","t1ce","t2","seg"]),
    Orientationd(keys=["flair","t1","t1ce","t2","seg"], axcodes="RAS"),
    Spacingd(keys=["flair","t1","t1ce","t2","seg"],
             pixdim=(1,1,1), mode=("bilinear","bilinear","bilinear","bilinear","nearest")),
    CropForegroundd(keys=["flair","t1","t1ce","t2","seg"], source_key="flair"),
    ResizeWithPadOrCropd(keys=["flair","t1","t1ce","t2","seg"], spatial_size=IMG_SIZE),
    NormalizeIntensityd(keys=["flair","t1","t1ce","t2"], nonzero=True, channel_wise=True),
    ConcatItemsd(keys=["flair","t1","t1ce","t2"], name="image"),
    EnsureTyped(keys=["image","seg"])
])

test_transforms = monai.transforms.Compose([
    LoadImaged(keys=["flair","t1","t1ce","t2"]),
    EnsureChannelFirstd(keys=["flair","t1","t1ce","t2"]),
    Orientationd(keys=["flair","t1","t1ce","t2"], axcodes="RAS"),
    Spacingd(keys=["flair","t1","t1ce","t2"], pixdim=(1,1,1),
             mode=("bilinear","bilinear","bilinear","bilinear")),
    CropForegroundd(keys=["flair","t1","t1ce","t2"], source_key="flair"),
    ResizeWithPadOrCropd(keys=["flair","t1","t1ce","t2"], spatial_size=IMG_SIZE),
    NormalizeIntensityd(keys=["flair","t1","t1ce","t2"], nonzero=True, channel_wise=True),
    ConcatItemsd(keys=["flair","t1","t1ce","t2"], name="image"),
    EnsureTyped(keys=["image"])
])


In [ ]:
train_ds = CacheDataset(train_files, transform=train_transforms, cache_rate=0.7)
test_ds  = Dataset(test_files, transform=test_transforms)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(train_ds, batch_size=1, shuffle=False)  # since training has labels
test_loader  = DataLoader(test_ds, batch_size=1, shuffle=False)

print("Batches => Train:", len(train_loader), "Val:", len(val_loader), "Test:", len(test_loader))


In [ ]:
model = SwinUNETR(
       
    in_channels=4,
    out_channels=4,
    feature_size=48,
    use_checkpoint=True
).to(device)


loss_dicece = DiceCELoss(include_background=False, to_onehot_y=True, softmax=True)
loss_tversky = TverskyLoss(alpha=0.3, beta=0.7, to_onehot_y=True, softmax=True)
loss_focal = FocalLoss(include_background=False, to_onehot_y=True, gamma=2.0)

def hybrid_loss(pred, target):
    target[target == 4] = 3
    l1 = loss_dicece(pred, target)
    l2 = loss_tversky(pred, target)
    l3 = loss_focal(pred, target)
    return 0.4*l1 + 0.3*l2 + 0.3*l3

optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-5)
scaler = torch.cuda.amp.GradScaler()


In [ ]:
def dice_region(pred, gt):
    pred = pred.cpu().numpy()[0]
    gt   = gt.cpu().numpy()[0]
    gt[gt==4] = 3

    eps = 1e-7
    scores = {}
    scores["WT"] = (2*np.sum((pred>0)&(gt>0))) / (np.sum(pred>0)+np.sum(gt>0)+eps)
    scores["TC"] = (2*np.sum(((pred==1)|(pred==3)) & ((gt==1)|(gt==3)))) / (
                   np.sum((pred==1)|(pred==3))+np.sum((gt==1)|(gt==3))+eps)
    scores["ET"] = (2*np.sum((pred==3)&(gt==3))) / (np.sum(pred==3)+np.sum(gt==3)+eps)
    return scores


In [ ]:
best_dice = 0
per_class_dice_history = []   
train_losses = []             

for epoch in range(1, EPOCHS+1):
    model.train()
    running_loss = 0

    # ================== TRAIN ==================
    pbar = tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS}")
    for batch in pbar:
        img = batch["image"].to(device)
        seg = batch["seg"].to(device)

        optimizer.zero_grad()
        with torch.amp.autocast('cuda'):
            pred = model(img)
            loss = hybrid_loss(pred, seg)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item()
        pbar.set_postfix({"loss": f"{running_loss/(pbar.n+1):.4f}"})

    # ================== VALIDATION ==================
    model.eval()
    dices = {"WT":[], "TC":[], "ET":[]}

    with torch.no_grad():
        vbar = tqdm(val_loader, desc="Validating", leave=False)
        for v in vbar:
            img = v["image"].to(device)
            seg = v["seg"].to(device)

            out = sliding_window_inference(img, IMG_SIZE, 1, model)
            pred = torch.argmax(out.softmax(1), 1, keepdim=True)
            d = dice_region(pred, seg)
            for k in d: dices[k].append(d[k])

    mean_dice = np.mean([np.mean(v) for v in dices.values()])
    print(f"🎯 Dice -> WT:{np.mean(dices['WT']):.4f}, TC:{np.mean(dices['TC']):.4f}, ET:{np.mean(dices['ET']):.4f}, Mean:{mean_dice:.4f}")

    # ---- Save epoch metrics ----
    per_class_dice_history.append([
        np.mean(dices["WT"]),
        np.mean(dices["TC"]),
        np.mean(dices["ET"])
    ])
    train_losses.append(running_loss/(len(train_loader)))

    if mean_dice > best_dice:
        best_dice = mean_dice
        torch.save(model.state_dict(), f"{CKPT_DIR}/best_swin_hybrid.pth")
        print("💾 Saved best model!")


In [ ]:
model.load_state_dict(torch.load(f"{CKPT_DIR}/best_swin_hybrid.pth", map_location=device))
model.eval()

for batch in tqdm(test_loader, desc="Inference"):
    pid = batch["id"][0]
    img = batch["image"].to(device)

    out = sliding_window_inference(img, IMG_SIZE, 1, model)
    pred = torch.argmax(out.softmax(1), 1).cpu().numpy()[0]

    affine = np.eye(4)
    for key in ["flair","t1","t1ce","t2"]:
        if key in batch and hasattr(batch[key][0], "meta"):
            try:
                affine = nib.load(batch[key][0].meta["filename_or_obj"]).affine
                break
            except: pass

    nib.save(nib.Nifti1Image(pred.astype(np.uint8), affine),
             f"{PRED_DIR}/{pid}_pred.nii.gz")


In [ ]:
# ===================== SAVE RESULTS =====================
import pandas as pd

df = pd.DataFrame(per_class_dice_history, columns=["WT","TC","ET"])
df["Mean"] = df.mean(axis=1)
df["Train_Loss"] = train_losses
df.to_csv(f"{OUTDIR}/metrics_log.csv", index_label="Epoch")

print("\n📌 Metrics saved at:", f"{OUTDIR}/metrics_log.csv")

# ===================== PLOT DICE CURVES =====================
plt.figure(figsize=(10,5))
plt.plot(df["WT"], label="WT")
plt.plot(df["TC"], label="TC")
plt.plot(df["ET"], label="ET")
plt.plot(df["Mean"], label="Mean", linewidth=3, color="black")
plt.title("Dice Score Progress")
plt.xlabel("Epoch")
plt.ylabel("Dice")
plt.legend()
plt.grid()
plt.savefig(f"{OUTDIR}/dice_curve.png", dpi=200)
plt.show()

# ===================== PLOT LOSS =====================
plt.figure(figsize=(6,4))
plt.plot(df["Train_Loss"], marker="o")
plt.title("Training Loss Curve")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.grid()
plt.savefig(f"{OUTDIR}/loss_curve.png", dpi=200)
plt.show()
